In [242]:
import numpy as np 
import pandas as pd 

import os
import string
from string import digits
import matplotlib.pyplot as plt
import re

import seaborn as sns
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.layers import LSTM, Input, Dense,Embedding, Concatenate, TimeDistributed
from keras.models import Model
from tensorflow.keras.utils import plot_model
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

In [243]:
lines=pd.read_csv("../input/test-2/eng-asm.csv",encoding='utf-8')

In [244]:
lines.isna().any()

english_sentence     False
assamese_sentence    False
dtype: bool

In [245]:
lines.drop_duplicates(inplace=True)

In [246]:
lines.head()

,english_sentence,assamese_sentence
0,Hi.,নমস্কাৰ।
1,Hi.,হাই।
2,Run!,দৌৰ!
3,Run!,দৌৰাঁ!
4,Run!,দৌৰক!


In [247]:
# Lowercase all characters
lines['english_sentence']=lines['english_sentence'].apply(lambda x: x.lower())
lines['assamese_sentence']=lines['assamese_sentence'].apply(lambda x: x.lower())

# Remove quotes
lines['english_sentence']=lines['english_sentence'].apply(lambda x: re.sub("'", '', x))

# Remove all the special characters
exclude = set(string.punctuation)
lines['english_sentence']=lines['english_sentence'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

# Remove all numbers from text
    #Remove english numbers
remove_digits = str.maketrans('', '', digits)
lines['english_sentence']=lines['english_sentence'].apply(lambda x: x.translate(remove_digits))
lines['assamese_sentence']=lines['assamese_sentence'].apply(lambda x: x.translate(remove_digits))
    # Remove assamese numbers
lines['assamese_sentence'] = lines['assamese_sentence'].apply(lambda x: re.sub("[০১২৩৪৫৬৭৮৯]", "", x))

# Remove white spaces
lines['english_sentence']=lines['english_sentence'].apply(lambda x: x.strip())
lines['assamese_sentence']=lines['assamese_sentence'].apply(lambda x: x.strip())
lines['english_sentence']=lines['english_sentence'].apply(lambda x: re.sub(" +", " ", x))
lines['assamese_sentence']=lines['assamese_sentence'].apply(lambda x: re.sub(" +", " ", x))

# Add start and end tokens to target sequences
lines['assamese_sentence'] = lines['assamese_sentence'].apply(lambda x : 'START_ '+ x + ' _END')

# Get lengths
lines['length_eng_sentence']=lines['english_sentence'].apply(lambda x:len(x.split(" ")))
lines['length_assm_sentence']=lines['assamese_sentence'].apply(lambda x:len(x.split(" ")))

In [248]:
lines.head()

,english_sentence,assamese_sentence,length_eng_sentence,length_assm_sentence
0,hi,START_ নমস্কাৰ। _END,1,3
1,hi,START_ হাই। _END,1,3
2,run,START_ দৌৰ! _END,1,3
3,run,START_ দৌৰাঁ! _END,1,3
4,run,START_ দৌৰক! _END,1,3


In [249]:
# Get English and Assamese Vocabulary
all_eng_words=set()
for eng in lines['english_sentence']:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)

all_assamese_words=set()
for assm in lines['assamese_sentence']:
    for word in assm.split():
        if word not in all_assamese_words:
            all_assamese_words.add(word)

In [250]:
print('Vocab size of english words:',len(all_eng_words))
print('Vocab size of assamese words:',len(all_assamese_words))

Vocab size of english words: 1218
Vocab size of assamese words: 2192


In [251]:
max_length_src=max(lines['length_eng_sentence'])
max_length_tar=max(lines['length_assm_sentence'])
print("maximum length of English Sentence:",max_length_src)
print("maximum length of Assamese Sentence:",max_length_tar)

maximum length of English Sentence: 15
maximum length of Assamese Sentence: 16


In [252]:
lines.head()

,english_sentence,assamese_sentence,length_eng_sentence,length_assm_sentence
0,hi,START_ নমস্কাৰ। _END,1,3
1,hi,START_ হাই। _END,1,3
2,run,START_ দৌৰ! _END,1,3
3,run,START_ দৌৰাঁ! _END,1,3
4,run,START_ দৌৰক! _END,1,3


In [253]:
lines.shape

(1745, 4)

In [254]:
print("maximum length of Assamese Sentence ",max(lines['length_assm_sentence']))
print("maximum length of English Sentence ",max(lines['length_eng_sentence']))

maximum length of Assamese Sentence  16
maximum length of English Sentence  15


In [255]:
max_length_src=max(lines['length_assm_sentence'])
max_length_tar=max(lines['length_eng_sentence'])

In [256]:
input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_assamese_words))
num_encoder_tokens = len(all_eng_words)
num_decoder_tokens = len(all_assamese_words)
num_encoder_tokens, num_decoder_tokens

(1218, 2192)

In [257]:
num_decoder_tokens += 1 #for zero padding


In [258]:
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])

In [259]:
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

In [260]:
c=0
for k,v in reverse_input_char_index.items():
    print(k,'->',v)
    c+=1
    if c==10:
        break

1 -> a
2 -> aah
3 -> able
4 -> aboard
5 -> about
6 -> above
7 -> abroad
8 -> absent
9 -> accept
10 -> accepted


In [261]:
c=0
for k,v in input_token_index.items():
    print(k,'->',v)
    c+=1
    if c==10:
        break

a -> 1
aah -> 2
able -> 3
aboard -> 4
about -> 5
above -> 6
abroad -> 7
absent -> 8
accept -> 9
accepted -> 10


In [262]:
lines = shuffle(lines)
lines.head()

,english_sentence,assamese_sentence,length_eng_sentence,length_assm_sentence
418,you cant scare me,START_ তুমি মোক ভয় দেখুৱাব নোৱাৰা। _END,4,7
967,how are we feeling today,START_ আজি আপুনি কেনে অনুভৱ কৰি আছে? _END,5,8
886,he will be back shortly,START_ তেওঁ সোনকালে উভতি আহিব। _END,5,6
135,were freaks,START_ আমি অদ্ভুত। _END,2,4
1608,i heard something fall to the ground,START_ মই মাটিত কিবা পৰা শুনিছোঁ। _END,7,7


In [263]:
X, y = lines['english_sentence'], lines['assamese_sentence']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
X_train.shape, X_test.shape

((1396,), (349,))

In [264]:
def generate_batch(X = X_train, y = y_train, batch_size = 128):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

In [265]:
latent_dim=256

In [266]:
# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [267]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [268]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [269]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_11 (InputLayer)           (None, None)         0                                            
__________________________________________________________________________________________________
input_12 (InputLayer)           (None, None)         0                                            
__________________________________________________________________________________________________
embedding_11 (Embedding)        (None, None, 256)    311808      input_11[0][0]                   
__________________________________________________________________________________________________
embedding_12 (Embedding)        (None, None, 256)    561408      input_12[0][0]                   
__________________________________________________________________________________________________
lstm_11 (L

In [270]:
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 128
epochs = 100

In [271]:
model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples//batch_size,
                    epochs=epochs,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples//batch_size)



Epoch 1/100
10/10 [==============================] - 4s 362ms/step - loss: 6.8769 - val_loss: 6.0654
Epoch 2/100
10/10 [==============================] - 1s 66ms/step - loss: 5.7784 - val_loss: 5.9746
Epoch 3/100
10/10 [==============================] - 1s 62ms/step - loss: 5.5337 - val_loss: 5.8298
Epoch 4/100
10/10 [==============================] - 1s 66ms/step - loss: 5.3949 - val_loss: 5.8783
Epoch 5/100
10/10 [==============================] - 1s 64ms/step - loss: 5.2854 - val_loss: 5.8329
Epoch 6/100
10/10 [==============================] - 1s 64ms/step - loss: 5.1945 - val_loss: 5.7552
Epoch 7/100
10/10 [==============================] - 1s 62ms/step - loss: 5.0926 - val_loss: 5.7835
Epoch 8/100
10/10 [==============================] - 1s 60ms/step - loss: 4.9747 - val_loss: 5.7457
Epoch 9/100
10/10 [==============================] - 1s 89ms/step - loss: 4.8811 - val_loss: 5.6853
Epoch 10/100
10/10 [==============================] - 1s 61ms/step - loss: 4.7864 - val_loss: 5.729

10/10 [==============================] - 1s 64ms/step - loss: 0.4681 - val_loss: 5.3396
Epoch 83/100
10/10 [==============================] - 1s 61ms/step - loss: 0.4531 - val_loss: 5.2303
Epoch 84/100
10/10 [==============================] - 1s 62ms/step - loss: 0.4329 - val_loss: 5.0232
Epoch 85/100
10/10 [==============================] - 1s 60ms/step - loss: 0.4230 - val_loss: 5.3560
Epoch 86/100
10/10 [==============================] - 1s 61ms/step - loss: 0.4018 - val_loss: 5.2258
Epoch 87/100
10/10 [==============================] - 1s 61ms/step - loss: 0.3879 - val_loss: 5.0044
Epoch 88/100
10/10 [==============================] - 1s 59ms/step - loss: 0.3718 - val_loss: 5.3481
Epoch 89/100
10/10 [==============================] - 1s 60ms/step - loss: 0.3592 - val_loss: 5.2316
Epoch 90/100
10/10 [==============================] - 1s 64ms/step - loss: 0.3502 - val_loss: 5.0074
Epoch 91/100
10/10 [==============================] - 1s 60ms/step - loss: 0.3302 - val_loss: 5.3426
Epo

In [272]:
# Encode the input sequence to get the "thought vectors"
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate prob dist. over the target vocabulary

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)


In [273]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [275]:
train_gen = generate_batch(X_train, y_train, batch_size = 1)
for k in range(-1,20):
    k+=1
    (input_seq, actual_output), _ = next(train_gen)
    decoded_sentence = decode_sequence(input_seq)
    print('Input English sentence:', X_train[k:k+1].values[0])
    print('Actual Assamese Translation:', y_train[k:k+1].values[0][6:-4])
    print('Predicted Assamese Translation:', decoded_sentence[:-4])
    print('*************************\n')

Input English sentence: youre to do as i tell you
Actual Assamese Translation:  মই কোৱামতে আপুনি কৰিব লাগিব। 
Predicted Assamese Translation:  মই কোৱামতে তুমি কৰিব লাগিব। 
*************************

Input English sentence: whats toms girlfriends name
Actual Assamese Translation:  টমৰ গাৰ্লফ্ৰেণ্ডৰ নাম কি? 
Predicted Assamese Translation:  টমৰ গাৰ্লফ্ৰেণ্ডৰ নাম কি? 
*************************

Input English sentence: tom shot a kid in the back
Actual Assamese Translation:  টমে সৰু ল'ৰা এটাৰ পিঠিত গুলি মাৰিলে। 
Predicted Assamese Translation:  টমে সৰু ল'ৰা এটাৰ পিঠিত গুলি মাৰিলে। 
*************************

Input English sentence: were baking cookies
Actual Assamese Translation:  আমি কুকিজ বেক কৰি আছোঁ। 
Predicted Assamese Translation:  আমি কুকিজ বেক কৰি আছোঁ। 
*************************

Input English sentence: you took the wrong key
Actual Assamese Translation:  তুমি ভুল চাবি লৈছা। 
Predicted Assamese Translation:  তুমি ভুল চাবি লৈছে। 
*************************

Input English sentence: h

In [ ]:
test_gen = generate_batch(X_test, y_test, batch_size = 1)
for k in range(-1,20):
    k+=1
    (input_seq, actual_output), _ = next(test_gen)
    decoded_sentence = decode_sequence(input_seq)
    print('Input English sentence:', X_test[k:k+1].values[0])
    print('Actual Assamese Translation:', y_test[k:k+1].values[0][6:-4])
    print('Predicted Assamese Translation:', decoded_sentence[:-4])
    print('*************************\n')